In [1]:
import pandas as pd
import pvlib
from datetime import datetime, timedelta, timezone
import pytz

from solarcalc.calculations import get_combined_data, get_combined_data_with_export_import, calculate_solar_totals, make_financial_projection
from solarcalc.database import get_lat_lon_from_eircode


In [2]:
get_lat_lon_from_eircode('E91 XW32')

(52.2924071, -7.723118599999999)

In [4]:
start = datetime(2024, 7, 11, tzinfo=timezone.utc)
end = datetime(2024, 7, 11, 23, 30, 0, tzinfo=timezone.utc)

df_comb = get_combined_data("8vwbkwzg", start, end)
df_comb

,datetime,power_solar,power_esb
0,2024-07-11 00:00:00+00:00,0.000000,382.0
1,2024-07-11 00:30:00+00:00,0.000000,466.0
2,2024-07-11 01:00:00+00:00,0.000000,1446.0
3,2024-07-11 01:30:00+00:00,0.000000,298.0
4,2024-07-11 02:00:00+00:00,0.000000,312.0
5,2024-07-11 02:30:00+00:00,0.000000,1310.0
6,2024-07-11 03:00:00+00:00,0.000000,522.0
7,2024-07-11 03:30:00+00:00,0.000000,1146.0
8,2024-07-11 04:00:00+00:00,0.000000,304.0
9,2024-07-11 04:30:00+00:00,60.904229,228.0


In [ ]:


start = datetime(2024, 1, 1, tzinfo=timezone.utc)
end = datetime(2024, 12, 31, 23, 30, 0, tzinfo=timezone.utc)


total_kwh_solar_used, total_kwh_export = calculate_solar_totals(df_comb)

# meeded variables
install_cost = 9000.0 # euros net of grants
import_tariff = 0.25 # euros per kwh
export_tariff = 0.15 # euros per kwh
market_interest_rate = 0.05 # 5% interest rate

projection_df = make_financial_projection(
    total_kwh_solar_used,
    total_kwh_export,
    install_cost,
    import_tariff,
    export_tariff,
    market_interest_rate
)

# Display the DataFrame
print(projection_df)



      Cum. Solar Savings  Cum. Export Revenue  Alt. Investment Value  \
Year                                                                   
1             564.187906           398.018047            9004.500000   
2            1128.375813           796.036094            9009.002250   
3            1692.563719          1194.054141            9013.506751   
4            2256.751625          1592.072188            9018.013505   
5            2820.939532          1990.090235            9022.522511   
6            3385.127438          2388.108282            9027.033773   
7            3949.315344          2786.126329            9031.547289   
8            4513.503251          3184.144376            9036.063063   
9            5077.691157          3582.162423            9040.581095   
10           5641.879063          3980.180470            9045.101385   
11           6206.066970          4378.198517            9049.623936   
12           6770.254876          4776.216564            9054.14

In [6]:
def calc_power_output(latitude, longitude, rated_power_per_panel, number_of_panels, panel_tilt, panel_azimuth, start, end):    
    location = pvlib.location.Location(latitude, longitude)
    #clearsky_irradiance =  location.get_clearsky(times)

    # Typical meteorogical year using the pvgis api
    data, _, _, _ = pvlib.iotools.get_pvgis_tmy(latitude=latitude, longitude=longitude, coerce_year=2024)
    data = data[(data.index >= start) & (data.index <= end)]
    solar_position = location.get_solarposition(data.index)

    #solar_position = location.get_solarposition(date_time)
    # Optimisic estimate of solar irradiation because it assumes clear skies
    # clear_sky = location.get_clearsky(date_time, model="ineichen")

    # Calculates the irradiance watts per m squared of the panel surface
    # Pvlib assumes a gorund albedo of around .2 which is correspondant with grass and soil, may differ for highly reflective surfaces like snow
    irradiance = pvlib.irradiance.get_total_irradiance(
        surface_tilt=panel_tilt,
        surface_azimuth=panel_azimuth,
        solar_zenith=solar_position['zenith'],
        solar_azimuth=solar_position['azimuth'],
        ghi=data['ghi'],
        dni=data['dni'],
        dhi=data['dhi'])
    
    #default values entered for wind_speed, u0, and u1
    module_temperature = pvlib.temperature.faiman(poa_global=irradiance["poa_global"], temp_air=data["temp_air"], wind_speed=1, u0=25.0, u1=6.84)

    dc_power = pvlib.pvsystem.pvwatts_dc(
    g_poa_effective=irradiance['poa_global'],
    # simplified assumptions that module and cell temperature are the same
    temp_cell=module_temperature,
    pdc0=rated_power_per_panel,
    gamma_pdc=-0.004,
    temp_ref=25.0,
    )
    #dc_power.index = dc_power.index.replace(year=2024)
    dc_power.index = dc_power.index.map(lambda x: x.replace(year=2024))

    # Plot x-axis: time, y-axis: dc_power on a line chart thingy
    #matplotlib.pyplot.plot
    

    return dc_power * number_of_panels

calc_power_output(48.47, -4.28, 445, 1, 40, 180, datetime(2024, 2, 28, 0, 0, 0, tzinfo=timezone.utc), datetime(2024, 2, 28, 23, 30, 0, tzinfo=timezone.utc))


time(UTC)
2024-02-28 00:00:00+00:00      0.000000
2024-02-28 01:00:00+00:00      0.000000
2024-02-28 02:00:00+00:00      0.000000
2024-02-28 03:00:00+00:00      0.000000
2024-02-28 04:00:00+00:00      0.000000
2024-02-28 05:00:00+00:00      0.000000
2024-02-28 06:00:00+00:00      0.000000
2024-02-28 07:00:00+00:00      0.000000
2024-02-28 08:00:00+00:00    100.488391
2024-02-28 09:00:00+00:00    167.502357
2024-02-28 10:00:00+00:00    287.673262
2024-02-28 11:00:00+00:00    328.629316
2024-02-28 12:00:00+00:00      0.000000
2024-02-28 13:00:00+00:00    381.367623
2024-02-28 14:00:00+00:00    344.918860
2024-02-28 15:00:00+00:00    284.434484
2024-02-28 16:00:00+00:00    157.240423
2024-02-28 17:00:00+00:00     33.500464
2024-02-28 18:00:00+00:00      0.000000
2024-02-28 19:00:00+00:00      0.000000
2024-02-28 20:00:00+00:00      0.000000
2024-02-28 21:00:00+00:00      0.000000
2024-02-28 22:00:00+00:00      0.000000
2024-02-28 23:00:00+00:00      0.000000
dtype: float64

In [246]:
#times_from_now = rounds_and_calculates_a_year_of_dates()

import numpy as np

clearsky_irradiance = calc_power_output(55.6846, 12.5389, 445, 1, 40, 180)



clearsky_irradiance = clearsky_irradiance.reset_index().rename(columns={"time(UTC)": "x", 0: "y"})

clearsky_irradiance = clearsky_irradiance[(clearsky_irradiance["x"] >= "2024-07-23T00:00:00.000") & (clearsky_irradiance["x"] < "2024-07-24T00:00:00.000")]

times = pd.date_range(start="2024-07-23 00:30:00.000", end="2024-07-24 00:00:00.000", freq="1h", tz="UTC")

new_rows = pd.DataFrame({"x": times, "y": np.nan})
    
clearsky_irradiance = pd.concat([clearsky_irradiance, new_rows], ignore_index=True)

clearsky_irradiance = clearsky_irradiance.sort_values(by="x").reset_index(drop=True)

if np.isnan(clearsky_irradiance.loc[0, "y"]):
    clearsky_irradiance.loc[0, "y"] = 0

if np.isnan(clearsky_irradiance.loc[len(clearsky_irradiance) - 1, "y"]):
    clearsky_irradiance.loc[len(clearsky_irradiance) - 1, "y"] = 0

def get_avg_value(values):
    values = np.array(values)
    for i in range(1, len(values)):
        if np.isnan(values[i]):
            values[i] = (values[i-1] + values[i+1]) / 2
    return values


#print(clearsky_irradiance.iloc["x"])

#clearsky_irradiance = clearsky_irradiance.to_json(orient="records", date_format="iso")

clearsky_irradiance["y"] = get_avg_value(clearsky_irradiance["y"].to_list())

clearsky_irradiance


,x,y
0,2024-07-23 00:00:00+00:00,0.000000
1,2024-07-23 00:30:00+00:00,0.000000
2,2024-07-23 01:00:00+00:00,0.000000
3,2024-07-23 01:30:00+00:00,0.000000
4,2024-07-23 02:00:00+00:00,0.000000
5,2024-07-23 02:30:00+00:00,0.000000
6,2024-07-23 03:00:00+00:00,0.000000
7,2024-07-23 03:30:00+00:00,1.673854
8,2024-07-23 04:00:00+00:00,3.347708
9,2024-07-23 04:30:00+00:00,10.407225


In [209]:
import pandas

hash_id = "krwm2n2y"
esb_intake = pandas.read_csv(f"/Users/michealmcmagh/Desktop/ise-entrance-submission-project/solarcalc/uploads/{hash_id}.csv")
esb_intake = pandas.concat([esb_intake["Read Date and End Time"], esb_intake["Read Value"]], join="inner", axis=1)
esb_intake = esb_intake.rename(columns={"Read Date and End Time": "x", "Read Value": "y"})
esb_intake["y"] = esb_intake["y"] * 1000
esb_intake["x"] = pandas.to_datetime(esb_intake["x"], dayfirst=True)
esb_intake = esb_intake.sort_values(by=['x'])
esb_intake = esb_intake[(esb_intake["x"] >= "2024-07-23T00:00:00.000") & (esb_intake["x"] < "2024-07-24T00:00:00.000")]


json_esb_intake = esb_intake.to_json(orient='records', date_format='iso')
json_esb_intake

'[{"x":"2024-07-23T00:00:00.000","y":494.0},{"x":"2024-07-23T00:30:00.000","y":440.0},{"x":"2024-07-23T01:00:00.000","y":598.0},{"x":"2024-07-23T01:30:00.000","y":360.0},{"x":"2024-07-23T02:00:00.000","y":298.0},{"x":"2024-07-23T02:30:00.000","y":290.0},{"x":"2024-07-23T03:00:00.000","y":1316.0},{"x":"2024-07-23T03:30:00.000","y":568.0},{"x":"2024-07-23T04:00:00.000","y":280.0},{"x":"2024-07-23T04:30:00.000","y":276.0},{"x":"2024-07-23T05:00:00.000","y":268.0},{"x":"2024-07-23T05:30:00.000","y":264.0},{"x":"2024-07-23T06:00:00.000","y":276.0},{"x":"2024-07-23T06:30:00.000","y":392.0},{"x":"2024-07-23T07:00:00.000","y":386.0},{"x":"2024-07-23T07:30:00.000","y":362.0},{"x":"2024-07-23T08:00:00.000","y":380.0},{"x":"2024-07-23T08:30:00.000","y":1014.0},{"x":"2024-07-23T09:00:00.000","y":280.0},{"x":"2024-07-23T09:30:00.000","y":264.0},{"x":"2024-07-23T10:00:00.000","y":1284.0},{"x":"2024-07-23T10:30:00.000","y":1086.0},{"x":"2024-07-23T11:00:00.000","y":852.0},{"x":"2024-07-23T11:30:00.00